# Lesson 4: Azure OpenAI Function Calling Feature

## Setup

**Note**: The pre-configured cloud resource grants you access to the Azure OpenAI GPT model. The key and endpoint provided below are intended for teaching purposes only. Your notebook environment is already set up with the necessary keys, which may differ from those used by the instructor during the filming.

In [2]:
import os
from openai import AzureOpenAI
import json

client = AzureOpenAI(
    api_version="2024-05-01-preview",
    azure_deployment="gpt-35-turbo",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key = os.getenv("AZURE_OPENAI_API_KEY"),
)

## 1. Using an illustrative example

In [3]:
def get_current_weather(location, unit="fahrenheit"):
    """Get the current weather in a given location. 
    The default unit when not specified is fahrenheit"""
    if "new york" in location.lower():
        return json.dumps(
            {"location": "New York", "temperature": "40", "unit": unit}
        )
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "50", "unit": unit}
        )
    elif "las vegas" in location.lower():
        return json.dumps(
            {"location": "Las Vegas", "temperature": "70", "unit": unit}
        )
    else:
        return json.dumps(
            {"location": location, "temperature": "unknown"}
        )

get_current_weather("New York")

'{"location": "New York", "temperature": "40", "unit": "fahrenheit"}'

### Define the tools

In [4]:
messages = [
    {"role": "user",
     "content": """What's the weather like in San Francisco,
                   New York, and Las Vegass?"""
    }
]

tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": """Get the current weather in a given
                              location.The default unit when not
                              specified is fahrenheit""",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": """The city and state,
                                        e.g. San Francisco, CA""",
                    },
                    "unit": {
                        "type": "string",
                        "default":"fahrenheit",
                        "enum": [ "fahrenheit", "celsius"],
                        "description": """The messuring unit for
                                          the temperature.
                                          If not explicitly specified
                                          the default unit is 
                                          fahrenheit"""
                    },
                },
                "required": ["location"],
            },
        },
    }
]

### Use the function calling

In [5]:
response = client.chat.completions.create(
    model="gpt-35-turbo",
    messages=messages,
    tools=tools,
    tool_choice="auto", 
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print (tool_calls)
    
    available_functions = {
        "get_current_weather": get_current_weather,
    } 
    messages.append(response_message)  
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_to_call = available_functions[function_name]
        function_args = json.loads(tool_call.function.arguments)
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )
        messages.append(
            {
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": function_response,
            }
        )  
    print (messages)

[ChatCompletionMessageToolCall(id='call_zT1AreLNyc9MqdrV7fDoVkPN', function=Function(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), type='function')]
[{'role': 'user', 'content': "What's the weather like in San Francisco,\n                   New York, and Las Vegass?"}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_zT1AreLNyc9MqdrV7fDoVkPN', function=Function(arguments='{\n  "location": "San Francisco, CA"\n}', name='get_current_weather'), type='function')]), {'tool_call_id': 'call_zT1AreLNyc9MqdrV7fDoVkPN', 'role': 'tool', 'name': 'get_current_weather', 'content': '{"location": "San Francisco", "temperature": "50", "unit": null}'}]


In [6]:
second_response = client.chat.completions.create(
            model="gpt-35-turbo",
            messages=messages,
        )
print (second_response)

ChatCompletion(id='chatcmpl-Aq0EzgnQFSSz1cdDFC5bSyW9251TQ', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The current weather in San Francisco, CA is 50°F.', role='assistant', function_call=None, tool_calls=None), content_filter_results={'hate': {'filtered': False, 'severity': 'safe'}, 'protected_material_code': {'filtered': False, 'detected': False}, 'protected_material_text': {'filtered': False, 'detected': False}, 'self_harm': {'filtered': False, 'severity': 'safe'}, 'sexual': {'filtered': False, 'severity': 'safe'}, 'violence': {'filtered': False, 'severity': 'safe'}})], created=1736957345, model='gpt-35-turbo', object='chat.completion', system_fingerprint=None, usage=CompletionUsage(completion_tokens=13, prompt_tokens=71, total_tokens=84), prompt_filter_results=[{'prompt_index': 0, 'content_filter_results': {'hate': {'filtered': False, 'severity': 'safe'}, 'jailbreak': {'filtered': False, 'detected': False}, 'self_harm': {'fil

## 2. Using our SQL database 

**Note**: To access the data locally, use the following code:

```
os.makedirs("data",exist_ok=True)
!wget https://covidtracking.com/data/download/all-states-history.csv -P ./data/
file_url = "./data/all-states-history.csv"
df = pd.read_csv(file_url).fillna(value = 0)
```

In [7]:
data_path = os.getenv("DATA_PATH")
csv_path = os.path.join(data_path, "all-states-history.csv")

In [8]:
from sqlalchemy import create_engine
import pandas as pd

df = pd.read_csv(csv_path).fillna(value = 0)
df

,date,state,death,hospitalized,negative,positive,recovered
0,3/7/2021,AK,305,1293.0,0.0,56886,0.0
1,3/7/2021,AL,10148,45976.0,1931711.0,499819,295690.0
2,3/7/2021,AR,5319,14926.0,2480716.0,324818,315517.0
3,3/7/2021,AS,0,0.0,2140.0,0,0.0
4,3/7/2021,AZ,16328,57907.0,3073010.0,826454,0.0
5,3/7/2021,CA,54124,0.0,0.0,3501394,0.0
6,3/7/2021,CO,5989,23904.0,2199458.0,436602,0.0
7,3/7/2021,CT,7704,0.0,0.0,285330,0.0
8,3/7/2021,DC,1030,0.0,0.0,41419,29570.0
9,3/7/2021,DE,1473,0.0,545070.0,88354,0.0


In [9]:
database_file_path = os.path.join(data_path, "test.db")

engine = create_engine(f'sqlite:///{database_file_path}')

df.to_sql(
    'all_states_history',
    con=engine,
    if_exists='replace',
    index=False)

21

### Create two functions

In [11]:
import json
import numpy as np
from sqlalchemy import text

# Define the functions
def get_hospitalized_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, state, hospitalized
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan

def get_positive_cases_for_state_on_date(state_abbr, specific_date):
    try:
        query = f"""
        SELECT date, state, positive AS positive_cases
        FROM all_states_history
        WHERE state = '{state_abbr}' AND date = '{specific_date}';
        """
        query = text(query)

        with engine.connect() as connection:
            result = pd.read_sql_query(query, connection)
        if not result.empty:
            return result.to_dict('records')[0]
        else:
            return np.nan
    except Exception as e:
        print(e)
        return np.nan

In [12]:
get_hospitalized_for_state_on_date("AK","3/7/2021")

{'date': '3/7/2021', 'state': 'AK', 'hospitalized': 1293.0}

In [13]:
get_positive_cases_for_state_on_date("AK","3/7/2021")

{'date': '3/7/2021', 'state': 'AK', 'positive_cases': 56886}

### Execute the function calling against the SQL database

In [14]:
# Define the messages and tools
messages = [
    {"role": "user",
     "content": """ how many hospitalized people we had in Alaska
                    the 3/7/2021?"""
    }
]

tools_sql = [
    {
        "type": "function",
        "function": {
            "name": "get_hospitalized_for_state_on_date",
            "description": """Retrieves the number of hospitalized people
                              for a specific state on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the state
                                          (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for
                                          the query in 'M/D/YYYY'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "get_positive_cases_for_state_on_date",
            "description": """Retrieves the number of positive cases
                              for a specific state on a specific date.""",
            "parameters": {
                "type": "object",
                "properties": {
                    "state_abbr": {
                        "type": "string",
                        "description": """The abbreviation of the 
                                          state (e.g., 'NY', 'CA')."""
                    },
                    "specific_date": {
                        "type": "string",
                        "description": """The specific date for the
                                          query in 'M/D/YYYY'
                                          format."""
                    }
                },
                "required": ["state_abbr", "specific_date"]
            }
        }
    }
]

response = client.chat.completions.create(
    model="gpt-35-turbo",
    messages=messages,
    tools=tools_sql,
    tool_choice="auto",
)

response_message = response.choices[0].message
tool_calls = response_message.tool_calls

if tool_calls:
    print(tool_calls)
    
    available_functions = {
        "get_positive_cases_for_state_on_date": get_positive_cases_for_state_on_date,
        "get_hospitalized_for_state_on_date": get_hospitalized_for_state_on_date
    }  
    messages.append(response_message)  
   
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        if function_name in available_functions:
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                state_abbr=function_args.get("state_abbr"),
                specific_date=function_args.get("specific_date"),
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": str(function_response),
                }
            ) 
        else:
            print(f"Function {function_name} is not available.")
    print(messages)

    second_response = client.chat.completions.create(
        model="gpt-35-turbo",
        messages=messages,
    )

# Organize and print the final response
final_message = second_response.choices[0].message.content
print("\nFinal Response:")
print(final_message)

[ChatCompletionMessageToolCall(id='call_Z2UyvhDrdkI54ejwT0rI44Ql', function=Function(arguments='{\n  "state_abbr": "AK",\n  "specific_date": "3/7/2021"\n}', name='get_hospitalized_for_state_on_date'), type='function')]
[{'role': 'user', 'content': ' how many hospitalized people we had in Alaska\n                    the 3/7/2021?'}, ChatCompletionMessage(content=None, role='assistant', function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_Z2UyvhDrdkI54ejwT0rI44Ql', function=Function(arguments='{\n  "state_abbr": "AK",\n  "specific_date": "3/7/2021"\n}', name='get_hospitalized_for_state_on_date'), type='function')]), {'tool_call_id': 'call_Z2UyvhDrdkI54ejwT0rI44Ql', 'role': 'tool', 'name': 'get_hospitalized_for_state_on_date', 'content': "{'date': '3/7/2021', 'state': 'AK', 'hospitalized': 1293.0}"}]

Final Response:
On March 7, 2021, there were 1,293 hospitalized people in Alaska.
